In [ ]:
from matplotlib import pyplot,image

shanthiPic = '/src/shanthiPictures/101ND750/DSC_0181.JPG'
img = image.imread(shanthiPic)
imgplot=pyplot.imshow(img)
pyplot.show()

In [ ]:
import hashlib
def calculateMD5Hash(file):
    fh = open(file, "rb")
    return hashlib.md5(fh.read()).hexdigest()
    

shanthiPic = '/src/data/testPythonDuplicateFinder/2015-09/IMG_1313.JPG'
print(calculateMD5Hash(shanthiPic))


In [ ]:
### List Extention ###
from os import walk as oswalk
from os.path import join as osjoin
from time import time
inputPath = '/src/data/testPythonDuplicateFinder'
startTime = time()
imageHashes = []
images =[]
for root, dirs, files in oswalk(inputPath):
    images = [osjoin(root, f) for f in files]
    imageHashes.extend([(image, calculateMD5Hash(image)) for image in images])

print('Took {}s for {} images'.format(time() - startTime, len(imageHashes)))
print ('images {}'.format(len(images)))


In [ ]:
import concurrent.futures
from os import walk as oswalk
from os.path import join as osjoin
from time import time
import hashlib
def calculateMD5Hash(file):
    fh = open(file, "rb")
    return hashlib.md5(fh.read()).hexdigest()
    
inputPaths = ['/src/data/testPythonDuplicateFinder']
duplicatesPath = '/src/data/duplicates'
inputImages = []
for inputPath in inputPaths:
    for root, dirs, files in oswalk(inputPath):
        inputImages.extend(img for img in [osjoin(root, f) for f in files] if img not in inputImages)


startTime = time()
# We can use a with statement to ensure threads are cleaned up promptly
with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    # Start the load operations and mark each future with its URL
    future_to_file = {executor.submit(calculateMD5Hash, inputFile): inputFile for inputFile in inputImages}
    imageHashMap = {}
    for future in concurrent.futures.as_completed(future_to_file):
        file = future_to_file[future]
        try:
            md5Hash = future.result()
            
        except Exception as exc:
            print('%r generated an exception: %s' % (file, exc))
        else:
            if md5Hash in imageHashMap: 
                imageHashMap[md5Hash].extend([file])
            else:
                imageHashMap[md5Hash] = [file]

print (imageHashMap)
print('Took {}s for  {} images'.format(time() - startTime, len(inputImages)))


In [ ]:
import json
imageHashMapJson = json.JSONEncoder(sort_keys=True, indent=0).encode(imageHashMap)
print (imageHashMapJson)
with open('/src/data/fileHashes.json', 'w') as jsonFile:
    jsonFile.write(imageHashMapJson)

#json.dump(imageHashMap, '/src/data/fileHashes.json')

# two ways to log an exception

In [77]:
## Two ways to log exception
import traceback
try:
    print (os.path.normpath('/data'))
except :
    print (traceback.format_exc())
    
import logging
try:
    print (os.path.normpath('/data'))
except :
    logging.exception("hello")
    
    

ERROR:root:hello
Traceback (most recent call last):
  File "<ipython-input-77-8eb620650385>", line 9, in <module>
    print (os.path.normpath('/data'))
NameError: name 'os' is not defined


Traceback (most recent call last):
  File "<ipython-input-77-8eb620650385>", line 3, in <module>
    print (os.path.normpath('/data'))
NameError: name 'os' is not defined



# Setup the data

In [187]:
%%bash
## Data Setup
rm -rf /src/data/test1/ /src/data/test2/ /src/data/duplicates
cp -r '/src/data/test1 (copy)/' '/src/data/test1/'
cp -r '/src/data/test2 (copy)/' '/src/data/test2/'



# Code starts here

In [188]:
## Start by loading the master list
## Prepare the list of unique file paths
## Find duplicates using the hash
## move duplicates to duplicates folder
## the run should output [hash, originalFilePath, [duplciate 1 path, /duplicate 2 path, duplicate 3,....]]
## save this master list along with the hash [hash, original file path]
import json
from os import walk, path, sep, makedirs

import concurrent.futures
import hashlib
import time
import shutil
import logging
import threading

def buildKnownFilesList(masterList):
    if (path.exists(masterList)):
        return json.load(open(masterList))
    else:
        return {}

def buildInputFilesList(inputPaths, knownImagesMap):
    inputImages = []
    for inputPath in inputPaths:
        for root, dirs, files in walk(inputPath):
            inputImages.extend(img for img in [path.join(root, f) for f in files] if img not in inputImages if img not in knownImagesMap.values())
    return inputImages

def calculateMD5Hash(file):
    fh = open(file, "rb")
    return hashlib.md5(fh.read()).hexdigest()

def checkForDuplicates(inputImages, knownImagesMap):
    duplicateImagesMap = {}
    # We can use a with statement to ensure threads are cleaned up promptly
    with concurrent.futures.ProcessPoolExecutor() as executor:
        # Start the load operations and mark each future with its URL
        future_to_file = {executor.submit(calculateMD5Hash, inputFile): inputFile for inputFile in inputImages}
        for future in concurrent.futures.as_completed(future_to_file):
            file = future_to_file[future]
            try:
                md5Hash = future.result()

            except Exception as exc:
                print('%r generated an exception: %s' % (file, exc))
            else:
                if md5Hash in knownImagesMap:
                    if md5Hash in duplicateImagesMap:
                        duplicateImagesMap[md5Hash].append(file)
                    else:
                        duplicateImagesMap[md5Hash] = [knownImagesMap[md5Hash], file]
                else:
                    knownImagesMap[md5Hash] = file
    return duplicateImagesMap

def moveFile(inputFile, duplicatesFolder):
    try:
        ## To join two absolute paths, we need to split the second path into its components and path
        ## that as an expanded list
        destFolder = path.join(duplicatesFolder, *path.normpath(path.dirname(inputFile)).split(sep))

        lock = threading.RLock()
        lock.acquire()
        ## Create the dest folder if necessary
        if (not path.exists(destFolder)):
            makedirs(destFolder)
        lock.release()

        shutil.move(inputFile, path.join(destFolder, path.basename(inputFile)))
        #print (inputFile)
    except:
        logging.exception("")
        raise
        
    
def moveTheDuplicates(duplicateImagesMap, duplicatesDestRoot):
    # We can use a with statement to ensure threads are cleaned up promptly
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Start the load operations and mark each future with its URL
        for duplicateFiles in duplicateImagesMap.values():
            if (len(duplicateFiles) > 1):
                future_to_file = {executor.submit(moveFile, duplicateFile, duplicatesDestRoot): duplicateFile for duplicateFile in duplicateFiles[1:]}
                for future in concurrent.futures.as_completed(future_to_file):
                    file = future_to_file[future]
            else:
                raise Exception("how did you get a single file entry in duplicate files list?")
             
            

def saveFileList(knownImagesMap, uniqueFilesMapPath, duplicateImagesMap, duplicateFilesMapPath):
    duplicateFilesMapJson = json.JSONEncoder(sort_keys=True, indent=0).encode(duplicateImagesMap)
    with open(duplicateFilesMapPath, 'w') as jsonFile:
        jsonFile.write(duplicateFilesMapJson)

   
    uniqueFilesMapJson = json.JSONEncoder(sort_keys=True, indent=0).encode(knownImagesMap)
    with open(uniqueFilesMapPath, 'w') as jsonFile:
        jsonFile.write(uniqueFilesMapJson)

def main():
    
    inputPaths=['/src/data/test1/', '/src/data/test2/']
    duplicatesPath='/src/data/duplicates'
    uniqueFilesMapPath='/src/data/uniqueFiles.json'
    duplicateFilesMapPath='/src/data/duplicateFiles.json'
    startTime = time.time()

    knownImagesMap=buildKnownFilesList(uniqueFilesMapPath)

    inputImages = buildInputFilesList(inputPaths, knownImagesMap)
    print (len(inputImages))
    duplicateImagesMap = checkForDuplicates(inputImages, knownImagesMap)
    print('checkForDuplicates took {}s'.format(time.time() - startTime))
    print ('inputImages length {}'.format(len(inputImages)))
    print ('knownImagesMap length {}'.format(len(knownImagesMap)))
    
    startTime = time.time()
    print()
    moveTheDuplicates(duplicateImagesMap, duplicatesPath)
    print('moveTheDuplicates took {}s'.format(time.time() - startTime))
    saveFileList(knownImagesMap, uniqueFilesMapPath, duplicateImagesMap, duplicateFilesMapPath)

main()

346
checkForDuplicates took 2.2411575317382812s
inputImages length 346
knownImagesMap length 423

moveTheDuplicates took 0.10678672790527344s
